In [ ]:
import matplotlib.pyplot as plt 
import numpy as np
import matplotlib.image as mpimg
import os
import scipy.misc as sm
import skimage

## Problem description

#### Input image loading

The input image is the set of 3 plates, corresponding to B, G, and R channels (top-down). You should implement the function $\tt{load}$\_$\tt{data}$ that reads the data and returns the list of images of plates.
$\tt{dir}$\_$\tt{name}$ is the path to the directory with plate images. If this directory is located in the same directory as this notebook, then default arguments can be used.

In [ ]:
def load_data(dir_name = 'faces_imgs'):    
    '''
    Load images from the "faces_imgs" directory
    Images are in JPG and we convert it to gray scale images
    '''
    imgs = []
    for filename in os.listdir(dir_name):
        if filename == '.DS_Store':
            continue
        if filename == "archive":
            continue
        img = mpimg.imread(dir_name + '/' + filename)
        #img = skimage.color.rgb2gray(img)
        imgs.append(img)
    return imgs
    
plates = load_data()

The dataset is a list of 2-dimensional arrays.

In [ ]:
# The auxiliary function `visualize()` displays the images given as argument.
def visualize(imgs, format=None):
    plt.figure(figsize=(20, 40))
    for i, img in enumerate(imgs):
        if img.shape[0] == 3:
            img = img.transpose(1,2,0)
        plt_idx = i+1
        plt.subplot(4, 2, plt_idx)    
        plt.imshow(img, vmin=0, vmax=255)
    plt.show()

visualize(plates)

#### The borders removal (1.5 points)
It is worth noting that there is a framing from all sides in most of the images. This framing can appreciably worsen the quality of channels alignment. Here, we suggest that you find the borders on the plates using Canny edge detector, and crop the images according to these edges. The example of using Canny detector implemented in skimage library can be found [here](http://scikit-image.org/docs/dev/auto_examples/edges/plot_canny.html).<br>

The borders can be removed in the following way:
* Apply Canny edge detector to the image.
* Find the rows and columns of the frame pixels. 
For example, in case of upper bound we will search for the row in the neighborhood of the upper edge of the image (e.g. 5% of its height). For each row let us count the number of edge pixels (obtained using Canny detector) it contains. Having these number let us find two maximums among them. Two rows corresponding to these maximums are edge rows. As there are two color changes in the frame (firstly, from light scanner background to the dark tape and then from the tape to the image), we need the second maximum that is further from the image border. The row corresponding to this maximum is the crop border. In order not to find two neighboring peaks, non-maximum suppression should be implemented: the rows next to the first maximum are set to zero, and after that, the second maximum is searched for.

#### Canny detector implementation (2.5 points)
You can write your own implementation of Canny edge detector to get extra points. <br>

Canny detection algorithm:
1. *Noise reduction.* To remove noise, the image is smoothed by Gaussian blur with the kernel of size $5 \times 5$ and $\sigma = 1.4$. Since the sum of the elements in the Gaussian kernel equals $1$, the kernel should be normalized before the convolution. <br><br>

2. *Calculating gradients.* When the image $I$ is smoothed, the derivatives $I_x$ and $I_y$ w.r.t. $x$ and $y$ are calculated. It can be implemented by convolving $I$ with Sobel kernels $K_x$ and $K_y$, respectively: 
$$ K_x = \begin{pmatrix} -1 & 0 & 1 \\ -2 & 0 & 2 \\ -1 & 0 & 1 \end{pmatrix}, K_y = \begin{pmatrix} 1 & 2 & 1 \\ 0 & 0 & 0 \\ -1 & -2 & -1 \end{pmatrix}. $$ 
Then, the magnitude $G$ and the slope $\theta$ of the gradient are calculated:
$$ |G| = \sqrt{I_x^2 + I_y^2}, $$
$$ \theta(x,y) = arctan\left(\frac{I_y}{I_x}\right)$$<br><br>

3. *Non-maximum suppression.* For each pixel find two neighbors (in the positive and negative gradient directions, supposing that each neighbor occupies the angle of $\pi /4$, and $0$ is the direction straight to the right). If the magnitude of the current pixel is greater than the magnitudes of the neighbors, nothing changes, otherwise, the magnitude of the current pixel is set to zero.<br><br>

4. *Double threshold.* The gradient magnitudes are compared with two specified threshold values, the first one is less than the second. The gradients that are smaller than the low threshold value are suppressed; the gradients higher than the high threshold value are marked as strong ones and the corresponding pixels are included in the final edge map. All the rest gradients are marked as weak ones and pixels corresponding to these gradients are considered in the next step.<br><br>

5. *Edge tracking by hysteresis.* Since a weak edge pixel caused from true edges will be connected to a strong edge pixel, pixel $w$ with weak gradient is marked as edge and included in the final edge map if and only if it is involved in the same blob (connected component) as some pixel $s$ with strong gradient. In other words, there should be a chain of neighbor weak pixels connecting $w$ and $s$ (the neighbors are 8 pixels around the considered one). You are welcome to make up and implement an algorithm that finds all the connected components of the gradient map considering each pixel only once.  After that, you can decide which pixels will be included in the final edge map (this algorithm should be single-pass, as well).

In [ ]:
from scipy import ndimage

def gaussian_kernel(size, sigma=1):
    size = int(size) // 2
    x, y = np.mgrid[-size:size+1, -size:size+1]
    normal = 1 / (2.0 * np.pi * sigma**2)
    g =  np.exp(-((x**2 + y**2) / (2.0*sigma**2))) * normal
    return g


def sobel_filters(img):
    Kx = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], np.float32)
    Ky = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]], np.float32)
    
    Ix = ndimage.filters.convolve(img, Kx)
    Iy = ndimage.filters.convolve(img, Ky)
    
    G = np.hypot(Ix, Iy)
    G = G / G.max() * 255
    theta = np.arctan2(Iy, Ix)
    
    return (G, theta)
    

def non_max_suppression(img, D):
    M, N = img.shape
    Z = np.zeros((M,N), dtype=np.int32)
    angle = D * 180. / np.pi
    #pi_4 = np.pi / 4
    #pi_2 = np.pi / 2
    angle[angle < 0] += 180

    
    for i in range(1,M-1):
        for j in range(1,N-1):
            try:
                #theta = D[i,j] #* 180 / np.pi #angle in degrees
                #theta_mod = theta % np.pi
                q = 255
                r = 255
                #alpha = None
                
               #angle 0
                if (0 <= angle[i,j] < 22.5) or (157.5 <= angle[i,j] <= 180):
                    q = img[i, j+1]
                    r = img[i, j-1]
                #angle 45
                elif (22.5 <= angle[i,j] < 67.5):
                    q = img[i+1, j-1]
                    r = img[i-1, j+1]
                #angle 90
                elif (67.5 <= angle[i,j] < 112.5):
                    q = img[i+1, j]
                    r = img[i-1, j]
                #angle 135
                elif (112.5 <= angle[i,j] < 157.5):
                    q = img[i-1, j-1]
                    r = img[i+1, j+1]

                """
                if (0 <= theta_mod < pi_4):
                    alpha = np.abs(np.tan(theta_mod))
                    q = (alpha * img[i + 1, j + 1]) + ((1 - alpha) * img[i, j + 1])
                    r = (alpha * img[i - 1, j - 1]) + ((1 - alpha) * img[i, j - 1]) 
                    
                elif (pi_4 <= theta_mod < pi_2):
                    alpha = np.abs(1./np.tan(theta_mod))
                    q = (alpha * img[i + 1, j + 1]) + ((1 - alpha) * img[i + 1, j])
                    r = (alpha * img[i - 1, j - 1]) + ((1 - alpha) * img[i - 1, j])
                    
                elif (pi_2 <= theta_mod < (3*pi_4)):
                    alpha = np.abs(1./np.tan(theta_mod))
                    q = (alpha * img[i + 1, j - 1]) + ((1 - alpha) * img[i + 1, j])
                    r = (alpha * img[i - 1, j + 1]) + ((1 - alpha) * img[i - 1, j])
                
                elif ((3*pi_4) <= theta_mod < np.pi):
                    alpha = np.abs(np.tan(theta_mod))
                    q = (alpha * img[i + 1, j - 1]) + ((1 - alpha) * img[i, j - 1])
                    r = (alpha * img[i - 1, j + 1]) + ((1 - alpha) * img[i, j + 1])
                """
                if (img[i,j] >= q) and (img[i,j] >= r):
                    Z[i,j] = img[i,j]
                else:
                    Z[i,j] = 0
                

            except IndexError as e:
                pass
    
    return Z

def threshold(img, lowThresholdRatio=0.05, highThresholdRatio=0.09):
    
    highThreshold = img.max() * highThresholdRatio;
    lowThreshold = highThreshold * lowThresholdRatio;
    
    M, N = img.shape
    res = np.zeros((M,N), dtype=np.int32)
    
    weak = np.int32(25)
    strong = np.int32(255)
    
    strong_i, strong_j = np.where(img >= highThreshold)
    zeros_i, zeros_j = np.where(img < lowThreshold)
    
    weak_i, weak_j = np.where((img <= highThreshold) & (img >= lowThreshold))
    
    res[strong_i, strong_j] = strong
    res[weak_i, weak_j] = weak
    
    return (res, weak, strong)

def hysteresis(img, weak, strong=255):
    
    M, N = img.shape  
    
    for i in range(1, M-1):
        for j in range(1, N-1):
            if (img[i,j] == weak):
                try:
                    if ((img[i+1, j-1] == strong) or (img[i+1, j] == strong) or (img[i+1, j+1] == strong)
                        or (img[i, j-1] == strong) or (img[i, j+1] == strong)
                        or (img[i-1, j-1] == strong) or (img[i-1, j] == strong) or (img[i-1, j+1] == strong)):
                        img[i, j] = strong
                    else:
                        img[i, j] = 0
                except IndexError as e:
                    pass
    
    return img

In [ ]:
#################################################################
# TODO: implement Canny detector yourself.                      #
#       You can use methods from scipy.ndimage if you need.     #
#################################################################
from  skimage.feature import canny
from scipy.ndimage import gaussian_filter
from scipy.ndimage.filters import convolve

from scipy import misc
import numpy as np


def Canny_detector(img):
    """ Your implementation instead of skimage """
    
    img_filtered = convolve(img, gaussian_kernel(5, sigma=1.4))
    grad, theta = sobel_filters(img_filtered)
    img_nms = non_max_suppression(grad, theta)
    img_thresh, weak, strong = threshold(img_nms, lowThresholdRatio=0.07, highThresholdRatio=0.19)
    img_final = hysteresis(img_thresh, weak, strong=strong)
   
    return img_final

In [ ]:
canny_imgs = []
for i, img in enumerate(plates):
    print("Processing image %i" % (i+1))
    canny_img = Canny_detector(img)
    canny_imgs.append(canny_img)
    
visualize(canny_imgs, 'gray')

In [ ]:
np.argsort(-np.sum(canny_imgs[0][0:22,:], axis=1))[1]
#canny_imgs[0][:,-57:].sum(axis=0)

In [ ]:

print(np.flip(np.sum(canny_imgs[0][:,-22:], axis=0), axis=0)/255)
print(np.argsort(-np.flip(np.sum(canny_imgs[0][:,-22:], axis=0), axis=0)))
print(np.argsort(-np.sum(canny_imgs[0][:,-22:-12], axis=0)))

In [ ]:
np.argsort(np.sum(canny_img[:,-44:], axis=0))

In [ ]:
#################################################################
# TODO: Implement the removal of the tape borders using         #
#       the output of Canny edge detector ("canny_img" list)    #
#################################################################

def remove_borders(img, canny_img):
    """ Your implementation instead of the following one"""   
    print(img.shape)
    ys, xs = img.shape
    dy = int(img.shape[0] * 0.05)
    dx = int(img.shape[1] * 0.1)
    print(dy, dx)

    y_indices = np.argsort(-np.sum(canny_img[0:dy,:], axis=-1))
    f_indices_upper_y = y_indices[0]
    indices_upper_y = y_indices[y_indices > f_indices_upper_y+1][0]
    
    y_indices = np.argsort(-np.flip(np.sum(canny_img[-dy:,:], axis=-1), axis=0))
    f_indices_lower_y = y_indices[0]
    indices_lower_y = y_indices[y_indices > f_indices_lower_y+1][0]
    
    
    
    x_indices = np.argsort(-np.sum(canny_img[:,0:dx], axis=0))
    f_indices_left_x = x_indices[0]
    if len(x_indices) == (f_indices_left_x + 1):
        indices_left_x = f_indices_left_x
    else:
        indices_left_x = x_indices[x_indices > f_indices_left_x+1][0]

    
    x_indices = np.argsort(-np.flip(np.sum(canny_img[:,-dx:], axis=0), axis=0))
    f_indices_right_x = x_indices[0]
    if len(x_indices) == (f_indices_right_x + 1):
        indices_right_x = f_indices_right_x
    else:
        indices_right_x = x_indices[x_indices > f_indices_right_x+1][0]
    
    
    #print(indices_upper_y , indices_lower_y, indices_left_x , indices_right_x)
    #print("new dims => ", img[indices_upper_y : -indices_lower_y, indices_left_x : -indices_right_x].shape)
    return img[indices_upper_y : -indices_lower_y, indices_left_x : -indices_right_x]



cropped_imgs = []
#crop borders
for i, img in enumerate(plates):
    cropped_imgs.append(remove_borders(img, canny_imgs[i]))

visualize(cropped_imgs, 'gray')

#### Channels separation  (0.5 points)

The next step is to separate the image into three channels (B, G, R) and make one colored picture. To get channels, you can divide each plate into three equal parts.

In [ ]:
#################################################################
# TODO: implement the function impose_components transforming   #
#       cropped black-and-white images cropped_imgs             #
#       into the list of RGB images rgb_imgs                    #
#################################################################
def impose_components(img):
    """ Your implementation """   
    h, w =img.shape
    new_h = h // 3
    img = img[:h-(h % new_h),:]
    new_img = np.zeros((h//3, w, 3), dtype=np.int32)
    new_img[:,:,2] = img[:new_h,:]
    new_img[:,:,1] = img[new_h:2*new_h,:]
    new_img[:,:,0] = img[2*new_h:,:]
    
    return new_img
    


rgb_imgs = []
for cropped_img in cropped_imgs:
    rgb_img = impose_components(cropped_img)
    rgb_imgs.append(rgb_img)

visualize(rgb_imgs)

#### Search for the best shift for channel alignment (1 point for metrics implementation + 2 points for channel alignment)

In order to align two images, we will shift one image relative to another within some limits (e.g. from $-15$ to $15$ pixels). For each shift, we can calculate some metrics in the overlap of the images. Depending on the metrics, the best shift is the one the metrics achieves the greatest or the smallest value for. We suggest that you implement two metrics and choose the one that allows to obtain the better alignment quality:

* *Mean squared error (MSE):*<br><br>
$$ MSE(I_1, I_2) = \dfrac{1}{w * h}\sum_{x,y}(I_1(x,y)-I_2(x,y))^2, $$<br> where *w, h* are width and height of the images, respectively. To find the optimal shift you should find the minimum MSE over all the shift values.
    <br><br>
* *Normalized cross-correlation (CC):*<br><br>
    $$
    I_1 \ast I_2 = \dfrac{\sum_{x,y}I_1(x,y)I_2(x,y)}{\sum_{x,y}I_1(x,y)\sum_{x,y}I_2(x,y)}.
    $$<br>
    To find the optimal shift you should find the maximum CC over all the shift values.

In [ ]:
#################################################################
# TODO: implement the functions mse и cor calculating           #
#       mean squared error and normalized cross-correlation     #
#       for two input images, respectively (1 point)            #
#################################################################
def mse(X, Y):
    xh, xw = X.shape
    yh, yw = Y.shape
    
  
    assert ((xw == yw) or (xh == yh)), \
    "Arguments dimensions are not equals: %s not equal to %s" % (str(xw), str(yw))
    a = [((X[i,j] - Y[i,j])**2) for i in range(xh) for j in range(xw)]
    return np.sum(a) / (xw * xh) 

def cor(X, Y):    
    xh, xw = X.shape
    yh, yw = Y.shape
    
    assert ((xw == yw) or (xh == yh)), \
    "Arguments dimensions are not equals: %s not equal to %s" % (str(xw), str(yw))
    #print(np.multiply(X, Y).shape)
    
    return np.sum(np.multiply(X, Y)) / (np.sum(X)*np.sum(Y))

In [ ]:
#################################################################
# TODO: implement the algorithm to find the best channel        #
#       shift and the alignment. Apply this algorithm for       #
#       rgb_imgs processing and get the final list of colored   #
#       pictures. These images will be used for the evaluation  #
#       of the quality of the whole algorithm.  (2 points)      #
#                                                               #
#       You can use the following interface or write your own.  #
#################################################################

def shift_img(img, shift=0):
    
    img_shifted = np.zeros(img.shape, dtype=np.uint8)
    
    if (shift < 0):
        img_shifted[shift:,:] = 0
        img_shifted[:shift,:] = img[-shift:,:]    
    elif (shift > 0):
        img_shifted[:shift,:] = 0
        img_shifted[shift:,:] = img[:-shift,:]
    else:
        img_shifted = img
    
    return img_shifted

def get_best_shift(channel1, channel2, shift_max=30):
    
    img_shifted = np.zeros((channel1.shape), dtype=np.uint8)
    
    best_mse, best_cor = +1e6, -1000
    best_shift_cor, best_shift_mse = None, None
    
    for shift in range(-shift_max, shift_max+1):
        img_shifted = shift_img(channel1, shift)
        
        mse_ = mse(img_shifted, channel2)
        cor_ = cor(img_shifted, channel2)
        
        if mse_ < best_mse:
            best_mse = mse_
            best_shift_mse = shift
        if cor_ > best_cor:
            best_cor = cor_
            best_shift_cor = shift
        
    return (best_shift_mse, best_mse, best_shift_cor, best_cor)

def get_best_image(rgb_img):
    
    shift_mse_1, _, shift_cor_1, _ = get_best_shift(rgb_img[:,:,0], rgb_img[:,:,1])
    shift_mse_2, _, shift_cor_2, _ = get_best_shift(rgb_img[:,:,2], rgb_img[:,:,1])
    
    new_img = rgb_img.copy()
    new_img[:,:,0] = shift_img(rgb_img[:,:,0], shift_mse_1)
    new_img[:,:,2] = shift_img(rgb_img[:,:,2], shift_mse_2)
    
    return new_img




In [ ]:
#Using MSE metrics
final_imgs = []
for img in rgb_imgs:
    final_img = get_best_image(img)
    final_imgs.append(final_img)

visualize(final_imgs)

In [ ]:
# Using COR Metrics
final_imgs = []
for img in rgb_imgs:
    final_img = get_best_image(img)
    final_imgs.append(final_img)

visualize(final_imgs)

# Face Alignment (2.5 points)

In this task, you have to implement face normalization and alignment. Most of the face images deceptively seem to be aligned, but since many face recognition algorithms are very sensitive to shifts and rotations, we need not only to find a face on the image but also normalize it. Besides, the neural networks usually used for recognition have fixed input size, so, the normalized face images should be resized as well.

There are six images of faces you have to normalize. In addition, you have the coordinates of the eyes in each of the pictures. You have to rotate the image so that the eyes are on the same height, crop the square box containing the face and transform it to the size $224\times 224.$ The eyes should be located symmetrically and in the middle of the image (on the height).

Here is an example of how the transformation should look like.

<img src = "https://cdn1.savepice.ru/uploads/2017/12/13/286e475ef7a4f4e59005bcf7de78742f-full.jpg">

#### Get data
You get the images and corresponding eyes coordinates for each person. You should implement the  function $\tt{load}$\_$\tt{faces}$\_$\tt{and}$\_$\tt{eyes}$ that reads the data and returns two dictionaries: the dictionary of images and the dictionary of eyes coordinates. Eyes coordinates is a list of two tuples $[(x_1,y_1),(x_2,y_2)]$.
Both dictionaries should have filenames as the keys.

$\tt{dir}$\_$\tt{name}$ is the path to the directory with face images, $\tt{eye}$\_$\tt{path}$ is the path to .pickle file with eyes coordinates. If these directory and file are located in the same directory as this notebook, then default arguments can be used.

In [ ]:
import pickle
import skimage

def load_faces_and_eyes(dir_name = 'faces_imgs', eye_path = './eyes.pickle'):
    
    eyes = {}
    imgs = {}
    for filename in os.listdir(dir_name):
        img = mpimg.imread(dir_name + '/' + filename)
        imgs[filename] = img
        
    eye_coords = pickle.load(open(eye_path, 'rb'))
    return imgs, eye_coords
    
    
faces, eyes = load_faces_and_eyes()

Here is how the input images look like:

In [ ]:
visualize(faces.values())

You may make the transformation using your own algorithm or by the following steps:
1. Find the angle between the segment connecting two eyes and horizontal line;
2. Rotate the image;
3. Find the coordinates of the eyes on the rotated image
4. Find the width and height of the box containing the face depending on the eyes coordinates
5. Crop the box and resize it to $224\times224$

In [ ]:
#################################################################
# TODO: implement the function transform_face that rotates      #
#       the image so that the eyes have equal ordinate,         #
#       crops the square box containing face and resizes it.    #
#       You can use methods from skimage library if you need.   #
#       (2.5 points)                                              #
#################################################################

def transform_face(image, eyes):
    (x1, y1), (x2, y2) = eyes

    a = np.arctan((y2-y1)/(x2-x1)) # angle in radians
    angle_deg = np.rad2deg(a) # angle in degrees
    
    rows, cols = image.shape[0], image.shape[1]
    center_x = (cols / 2 - 0.5)
    center_y =  rows / 2 - 0.5
    
    
    #Rotation of the image
    image = skimage.transform.rotate(image, angle_deg)
    
    # calculating the coordinates in the center of the image referential
    #since the rotation is done centered on the center of the image
    x1 = x1 - center_x
    y1 = y1 - center_y
    x2 = x2 - center_x
    y2 = y2 - center_y

    # calculate new coordinates
    newX1 = x1 * cos(a) + y1 * sin(a)
    newY1 = y1 * cos(a) - x1 * sin(a)
    newX2 = x2 * cos(a) + y2 * sin(a)
    newY2 = y2 * cos(a) - x2 * sin(a)
    
    # calculate coordonate in the original referential
    newX1 = newX1 + center_x
    newY1 = newY1 + center_y
    
    newX2 = newX2 + center_x
    newY2 = newY2 + center_y
    
    
    # cropping the image
    new_size = np.int32(224)
    new_size_center = new_size // 2

    middle_eyes_x = np.int(newX1 + ((newX2 - newX1) // 2))
    middle_eyes_y = np.int(newY1)
    
    start_x = max(0, middle_eyes_x - new_size_center)
    end_x = middle_eyes_x + new_size_center
    
    start_y = max(0, middle_eyes_y-new_size_center)
    end_y = middle_eyes_y + new_size_center
    
    
    return image[start_y:end_y, start_x:end_x]

In [ ]:
transformed_imgs = []
for i in faces:
    img = faces[i]
    eye = eyes[i]
    transformed = transform_face(img, eye)
    transformed_imgs.append(transformed)
    
visualize(transformed_imgs)